Notes for further improvements:

- flash attention: https://huggingface.co/google/paligemma-3b-pt-224/discussions/15
- As said in the paper: "Finally, we eased the multi-task learning by us- ing task-prefixes [14]". It doesnt help VQA but drastically helps captioning. It can also be considered as an improvement later on.
- There are important notes and research gaps in Appendix A: "A. More related work"

# Setup & Hugginface Connection

In [1]:
!pip install -q -U bitsandbytes

In [2]:
!pip install -q -U transformers datasets peft bitsandbytes

In [3]:
!pip install wandb

In [4]:
import transformers

In [5]:
import torch
import requests
from PIL import Image
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration

In [6]:
from huggingface_hub import login
login(token = "...")

## Model Inference Trial

In [6]:
pretrained_model_id = "google/paligemma-3b-pt-224"
processor = AutoProcessor.from_pretrained(pretrained_model_id)
pretrained_model = PaliGemmaForConditionalGeneration.from_pretrained(pretrained_model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [7]:
prompt = "What is behind the cat?"
image_file = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/cat.png?download=true"
raw_image = Image.open(requests.get(image_file, stream=True).raw)
inputs = processor(raw_image.convert("RGB"), prompt, return_tensors="pt")
output = pretrained_model.generate(**inputs, max_new_tokens=20)
print(processor.decode(output[0], skip_special_tokens=True)[len(prompt):])

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.



antique


In [8]:
prompt = "caption en"
image_file = "https://datasets-server.huggingface.co/assets/caglarmert/full_riscm/--/a955412d7370c9483142ee3ed8c77333f92fffa9/--/default/train/0/image/image.jpg?Expires=1747538771&Signature=q7CvXLvxbn-kCtXuJL5oqDBwFgHK2LJH63I7LM3LANAHklDqbSqz8hlx~WR0QsPIIL0jF5OnQm7XY3xz4cQZYQ62oJW6V7vvZamYZZgO33RWwRkqf70upOgc3UiS4cWA5BHws5ulT4wX8AbaO7g7eboo2PNpIdkoE385Sj0~Vsqct9NerDSeHMmSO2qD8QzIAsdQgBRmsbDW2~qJ6XxisSOYx2MJxQjy2GpQ8RRAEBBKkUPerzSCKKG1Cr-9lvuvCB-xyVH-~-mWMcZlbTi8BkAS1YsFYn2Qr8cofqzqPelrsM9LzTMWxjNwy6VYs102VEDUcrcmXy0jVitcZSMYkw__&Key-Pair-Id=K3EI6M078Z3AC3"
raw_image = Image.open(requests.get(image_file, stream=True).raw)
inputs = processor(raw_image.convert("RGB"), prompt, return_tensors="pt")
output = pretrained_model.generate(**inputs, max_new_tokens=20)
print(processor.decode(output[0], skip_special_tokens=True)[len(prompt):])

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.



the aircraft is seen from above .


In [11]:
prompt = "caption en"
image_file = "https://datasets-server.huggingface.co/assets/caglarmert/full_riscm/--/a955412d7370c9483142ee3ed8c77333f92fffa9/--/default/train/0/image/image.jpg?Expires=1747538771&Signature=q7CvXLvxbn-kCtXuJL5oqDBwFgHK2LJH63I7LM3LANAHklDqbSqz8hlx~WR0QsPIIL0jF5OnQm7XY3xz4cQZYQ62oJW6V7vvZamYZZgO33RWwRkqf70upOgc3UiS4cWA5BHws5ulT4wX8AbaO7g7eboo2PNpIdkoE385Sj0~Vsqct9NerDSeHMmSO2qD8QzIAsdQgBRmsbDW2~qJ6XxisSOYx2MJxQjy2GpQ8RRAEBBKkUPerzSCKKG1Cr-9lvuvCB-xyVH-~-mWMcZlbTi8BkAS1YsFYn2Qr8cofqzqPelrsM9LzTMWxjNwy6VYs102VEDUcrcmXy0jVitcZSMYkw__&Key-Pair-Id=K3EI6M078Z3AC3"
raw_image = Image.open(requests.get(image_file, stream=True).raw)
inputs = processor(raw_image.convert("RGB"), prompt, return_tensors="pt")
output = pretrained_model.generate(**inputs, max_new_tokens=20)
print(processor.decode(output[0], skip_special_tokens=True)[len(prompt):])

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.



the aircraft is seen from above .


# Model Finetunning

https://huggingface.co/blog/paligemma#using-transformers-1

In [7]:
import torch
import requests
from PIL import Image
from datasets import load_dataset
from peft import get_peft_model, LoraConfig
from transformers import Trainer
from transformers import TrainingArguments
from transformers import PaliGemmaProcessor
from transformers import BitsAndBytesConfig
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration

Suggested hyper-parameters in paper:

"The hyper-parameters we modify per-task are the following, in decreasing order of importance:
- Resolution (i.e. checkpoint): 224, 448, 896.
- Epochs: 1, 3, 10, 30, 100.
- Learning-rate: 3e-5, 1e-5, 3e-6.
- Label-smoothing: 0.0, 0.1, 0.3.
- Dropout in the LLM: 0.0, 0.1, 0.3.
- Weight decay: 0.0 or 0.1 × learning-rate. • Freeze ViT: false, true.
- Beam-search may benefit captioning.
The above are typical values we suggest exploring, with the recommended initial attempt value in bold."

Suggested values are presented in Appendix J - page 48.


Also, one setup is suggested as follows:

"However one question a reader may have is whether the choice of transfer hyper-parameter is important. To ablate that we run all tasks at 224px and under a single and simple hyper- parameter setup, which was also highlighted in bold in Section 3.2.4: lr=1e-5, bs=256, no dropout, no label smoothing, no weight decay, and not freezing anything. The only task depen- dent parameter is the number of epochs for which we use each task’s best one but cap it at 10."

I believe they can be partially transfered to QLora.



## Dataset

In [8]:
ds = load_dataset('caglarmert/full_riscm')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
ds

DatasetDict({
    train: Dataset({
        features: ['image', 'caption_1', 'caption_2', 'caption_3', 'caption_4', 'caption_5'],
        num_rows: 44521
    })
})

In [10]:
ds_dataframe = ds['train'].to_pandas()
ds_dataframe

,image,caption_1,caption_2,caption_3,caption_4,caption_5
0,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,A gray plane on the runway and the lawn beside .,A grey plane is on the runway by the lawn .,There is an airplane on the runway with a larg...,A plane is parked on the runway next to the gr...,There is a plane on the runway beside the grass .
1,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,Three small planes parked in a line on the air...,"There are four aircraft on the open ground, Th...",There are many planes of different sizes in a ...,Four planes are parked on the runway .,Four planes of different sizes were on the mar...
2,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,A plane parked in a line on the airport with s...,A white plane was parked on the instruction li...,An airplane parked in an open area with many c...,A plane is parked on the open space .,There is 1 plane on the ground marked .
3,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,A small plane and a big plane parked next to b...,A white plane and a gray plane parked at the b...,Two planes of different sizes are neatly parke...,A large plane and a small plane are parked nea...,Two planes are on the marked ground .
4,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,Two planes parked next to boarding bridges .,Two aircraft were parked at the departure gates .,Two planes of different sizes are neatly parke...,Two planes are parked next to the terminal .,Two planes are on the marked ground .
...,...,...,...,...,...,...
44516,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,A small tennis court with some plants and buil...,A tennis court is surrounded by some trees and...,There is a small tennis court with some buildi...,There is a tennis court surrounded by some tre...,There is a small tennis court and surrounded b...
44517,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,Two tennis courts arranged neatly with some pl...,Two tennis courts are surrounded by some trees...,There are two tennis courts surrounded by some...,There are two tennis courts surrounded by some...,There are two tennis courts arranged neatly an...
44518,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,A small tennis court with some plants and a ro...,A tennis court is surrounded by some trees and...,There is a small tennis court with a road besi...,There is a tennis court surrounded by some hou...,There is a small tennis court and surrounded b...
44519,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,A small tennis court with some plants and a sw...,A tennis court is surrounded by some trees and...,There is a small tennis court with a swimming ...,There is a tennis court surrounded by some car...,There is a small tennis court with some cars a...


In [11]:
# Since the HF version of the dataset is not loaded with split information, I split myself.
# Original data has 80 train, 10 test, 10 val split

# Using original split will be much better in final phase.

split_ds = ds["train"].train_test_split(test_size=0.2)

train_ds = split_ds["train"]
test_val_ds = split_ds["test"]

test_val_ds = test_val_ds.train_test_split(test_size=0.5)

test_ds = test_val_ds["train"]
val_ds = test_val_ds["test"]

print(f"Train dataset size: {len(train_ds)}")
print(f"Test dataset size: {len(test_ds)}")
print(f"Validation dataset size: {len(val_ds)}")

Train dataset size: 35616
Test dataset size: 4452
Validation dataset size: 4453


## Model

In [14]:
device = "cuda"
model_id = "google/paligemma-3b-pt-224"

# google/paligemma-3b-mix-224 -> mix. gives close to task spesific fine-tuned results. (Appendix H)
# google/paligemma-3b-ft-cococap-224-jax, google/paligemma-3b-ft-cococap-224
# google/paligemma-3b-ft-rsvqa-hr-224, google/paligemma-3b-ft-rsvqa-lr-224
  # B.23. RSVQA-hr: VQA for remote sensing (high res)
# google/paligemma-3b-ft-textcaps-224

We can try different checkpoints that are avialable in HF as explained in the paper:

"To show the effectiveness of the base mod- els, we transfer them to a wide range of indi- vidual academic benchmarks, using a simple uni- fied transfer recipe with few hyper-parameters. And to showcase the versatility beyond academic tasks, we also provide a “mix” transfer checkpoint, which transfers to a subset of these tasks at the same time, along with detailed captioning and long question-answering data. While this is not instruction tuning, it is a step in that direction."

In [21]:
# processor = PaliGemmaProcessor.from_pretrained(model_id)
processor = PaliGemmaProcessor.from_pretrained(model_id, use_fast=True)
def collate_fn(examples):
      # texts = ["<image> <bos> describe this image.}" for example in examples]
      texts = ["<image> <bos> caption en}" for example in examples] # as stated in original paper for captioning tasks
      labels= [example['caption_1'] for example in examples] # TO-DO -> using only one caption is not sufficient, we can leverage all of them as seperate examples maybe
      images = [example["image"].convert("RGB") for example in examples]
      tokens = processor(text=texts, images=images, suffix=labels,
               return_tensors="pt", padding="longest")
      tokens = tokens.to(torch.bfloat16).to(device)

      return tokens

As stated in "5.4. To freeze or not to freeze?" part of the PaliGemma paper:

- Frozing LM hurts the finetunning performance a lot.
- Vision encoders are generally kept frozen, however, when it is also fine-tuned it helped the performance a little bit.
- Resetting is not a option.
- No ablation is provided about linear projection layer tunning part.

This ablations can be applied to fine-tunnign with QLora or LoRa i guess.

In [34]:
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.bfloat16).to(device)
for param in model.vision_tower.parameters():
      param.requires_grad = False
for param in model.multi_modal_projector.parameters():
      param.requires_grad = False

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [35]:
model

PaliGemmaForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(256, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-26): 27 x SiglipEncoderLayer(
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (self_attn): SiglipAttention(
              (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm2): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): SiglipMLP(
          

### QLoRa

In [12]:
bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
)
lora_config = LoraConfig(
      r=8,

      # target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
      target_modules=["q_proj", "k_proj", "v_proj", # for  siglip and gemma
                      "o_proj", "gate_proj", "up_proj", "down_proj", # for gemma only
                      "out_proj", "fc1", "fc2", # for siglip only
                      "linear", # for multi-modal projector only
                      ],
      # lora_alpha=32,
      # lora_dropout=0.05,
      # bias="none",

      task_type="CAUSAL_LM",
)

Note from the paper:

"Model parameters and optimizer state are kept in float32 to guarantee stable training, but we verified that inference works just as well with bfloat16 model parameters."

In [15]:
# All
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 14,179,072 || all params: 2,937,645,552 || trainable%: 0.4827


In [21]:
# self-attention parts only (missing out_proj for siglip)
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 11,298,816 || all params: 2,934,765,296 || trainable%: 0.3850


In [38]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PaliGemmaForConditionalGeneration(
      (vision_tower): SiglipVisionModel(
        (vision_model): SiglipVisionTransformer(
          (embeddings): SiglipVisionEmbeddings(
            (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
            (position_embedding): Embedding(256, 1152)
          )
          (encoder): SiglipEncoder(
            (layers): ModuleList(
              (0-26): 27 x SiglipEncoderLayer(
                (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
                (self_attn): SiglipAttention(
                  (k_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=1152, out_features=1152, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Identity()
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=11

## Training Arguments

Reference:

- https://pyimagesearch.com/2024/12/02/fine-tune-paligemma-with-qlora-for-visual-question-answering/
- https://blog.roboflow.com/fine-tune-paligemma-2/

In [16]:
project_name = "paligemma-project-p2"
experiment_name = "qlora_paligemma_"

In [19]:
args=TrainingArguments(
      seed=42,
      remove_unused_columns=False,

      num_train_epochs=2,
      per_device_train_batch_size=4,
      gradient_accumulation_steps=4, # 8, 16
      warmup_steps=2,
      learning_rate=2e-5,
      weight_decay=1e-6,
      adam_beta2=0.999,
      logging_steps=10,
      optim="adamw_8bit", # paged_adamw_8bit

      save_strategy="steps",
      save_steps=10,
      push_to_hub=True,
      save_total_limit=1,
      output_dir=experiment_name,

      bf16=True,
      dataloader_pin_memory=False,

      report_to=["tensorboard", "wandb"]
)

In [18]:
import wandb

wandb.init(
    project=project_name,
    name=experiment_name,
    config={
        "learning_rate": args.learning_rate,
        "epochs": args.num_train_epochs,
        "batch_size": args.per_device_train_batch_size,
        "model_checkpoint": model_id,
        "dataset": "RISCM",

        "qlora_rank": 8,
        "qlora_alpha": 32,
        "qlora_dropout": 0.05,

        "frozen_image_encoder": False,
        "frozen_linear_projection": False,
        "frozen_text_decoder": False
    }
)

wandb: Currently logged in as: utkuden (utkuden-metu-middle-east-technical-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
trainer = Trainer(
      model=model,
      train_dataset=train_ds,
      eval_dataset=val_ds,
      data_collator=collate_fn,
      args=args
)
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,16.196400
20,15.408400
30,14.891100
40,13.981900
50,12.998900
60,12.288900
70,11.342700
80,10.790600
90,10.351400
100,10.234800


In [ ]:
trainer = Trainer(
      model=model,
      train_dataset=train_ds,
      data_collator=collate_fn,
      args=args
)
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
2,17.465000
4,15.938900
6,16.993700
8,15.160200
10,15.750600
12,15.279000
14,15.568500
16,14.970100
18,15.390400
20,14.663100


In [ ]:
trainer.push_to_hub()

# Inference

In [ ]:
pretrained_model_id = "google/paligemma-3b-pt-224"
finetuned_model_id = "caglarmert/finetuned_paligemma_riscm_small"
processor = AutoProcessor.from_pretrained(pretrained_model_id)
model = PaliGemmaForConditionalGeneration.from_pretrained(finetuned_model_id)

In [ ]:
prompt = "Describe this image."
image_file = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/cat.png?download=true"
raw_image = Image.open(requests.get(image_file, stream=True).raw)
inputs = processor(raw_image.convert("RGB"), prompt, return_tensors="pt")
output = model.generate(**inputs, max_new_tokens=40)
print(processor.decode(output[0], skip_special_tokens=True)[len(prompt):])

In [ ]:
prompt = "Describe this image."
image_file = test_ds[1]["image"]
inputs = processor(image_file, prompt, return_tensors="pt")
output = model.generate(**inputs, max_new_tokens=20)
print(processor.decode(output[0], skip_special_tokens=True)[len(prompt):])

# Evaluation

- Use CIDEr for evaluation
- Use train + val after finding the best hyperparameter and setup

In [ ]:
from evaluate import list_evaluation_modules
list_evaluation_modules(module_type="metric")

## Pre-trained

## Ours